<a href="https://colab.research.google.com/github/2025-01-sookmyung-opensource/Quiz/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# cnn 실습

In [ ]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical

import numpy as np
from PIL import Image
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
# Colab에서 labels.txt 업로드
from google.colab import files
uploaded = files.upload()  # labels.txt 선택

In [ ]:
# 이미지 폴더 업로드
from google.colab import files
import os

# 여러 개의 이미지 파일 업로드
uploaded = files.upload()  # Ctrl+A로 1.png~N.png 전체 선택

# images 폴더 생성 및 이동
os.makedirs("images", exist_ok=True)

for fname in uploaded.keys():
    os.rename(fname, os.path.join("images", fname))

In [ ]:
from tqdm import tqdm
from PIL import Image
import numpy as np
import os
from keras.utils import to_categorical
from keras import backend as K

# 이미지 설정
img_rows, img_cols = 28, 28
num_classes = 10
batch_size = 128
epochs = 12

# 라벨 파일 읽기
with open("labels.txt", "r") as f:
    labels = [int(line.strip()) for line in f.readlines()]

# 이미지와 라벨 매칭
image_dir = "images"
x_data = []
y_data = []

for i, label in tqdm(enumerate(labels), total=len(labels)):
    img_path = os.path.join(image_dir, f"{i+1}.png")
    if not os.path.exists(img_path):
        print(f"경고: {img_path} 파일이 없습니다. 건너뜁니다.")
        continue  # 파일 없으면 다음으로
    img = Image.open(img_path).convert("L").resize((img_rows, img_cols))  # 흑백 변환 및 크기 조정
    img_array = np.array(img) / 255.0
    x_data.append(img_array)
    y_data.append(label)

x_data = np.array(x_data)
y_data = np.array(y_data)

# 채널 차원 추가
if K.image_data_format() == 'channels_first':
    x_data = x_data.reshape(x_data.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_data = x_data.reshape(x_data.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# 원-핫 인코딩
y_data = to_categorical(y_data, num_classes)

100%|██████████| 10000/10000 [00:01<00:00, 7261.83it/s]


In [ ]:
# 학습/검증 분할
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42
)

# CNN 모델 구성
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# 컴파일 및 학습
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adadelta(),
    metrics=['accuracy']
)

model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test)
)

In [ ]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

print("y_train example:", np.argmax(y_train[:10], axis=1))
print("y_test example:", np.argmax(y_test[:10], axis=1))
print("x_train pixel range:", x_train.min(), "~", x_train.max())

In [ ]:
# 평가
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# 테스트 데이터 업로드
uploaded = files.upload()

# 업로드된 이미지 파일명
test_img_path = "1.png"

# 이미지 불러오기 및 전처리
img = Image.open(test_img_path).convert("L").resize((img_rows, img_cols))
img_array = np.array(img) / 255.0

if K.image_data_format() == 'channels_first':
    img_array = img_array.reshape(1, 1, img_rows, img_cols)
else:
    img_array = img_array.reshape(1, img_rows, img_cols, 1)

# 예측
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)[0]

print(f"예측 결과: {predicted_class}")